In [216]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import DBSCAN

In [217]:
data = pd.read_csv("post_Cleaning_EDA.csv")
print("after" + str(data.shape))
data['severity'] = data['severity'] - 1
#data = data[data.severity != 0]

indicies_2 = data[data.severity == 2].index
indicies_1 = data[data.severity == 1].index
indicies_0 = data[data.severity == 0].index

print(indicies_2)
print(indicies_1)
print(indicies_0)

sample_size = sum(data.severity == 2)
random_indices_0 = np.random.choice(indicies_0, sample_size, replace=False)
random_indices_1 = np.random.choice(indicies_1, sample_size, replace=False)
random_indices_2 = np.random.choice(indicies_2, sample_size, replace=False)


data1 = data.loc[random_indices_0]
data2 = data.loc[random_indices_1]
data3 = data.loc[random_indices_2]

#data = pd.concat([data1, data2, data3])
print(data)

print("after" + str(data.shape))

after(287947, 13)
Int64Index([   348,   1766,   1868,   2293,   2682,   2744,   2812,   3570,
              4326,   5536,
            ...
            287009, 287069, 287282, 287358, 287382, 287455, 287533, 287629,
            287888, 287934],
           dtype='int64', length=1214)
Int64Index([   142,    163,    326,    395,    469,    496,    500,    542,
               726,    759,
            ...
            287867, 287868, 287882, 287885, 287889, 287892, 287897, 287933,
            287944, 287946],
           dtype='int64', length=16280)
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            287935, 287936, 287937, 287938, 287939, 287940, 287941, 287942,
            287943, 287945],
           dtype='int64', length=270453)
        severity  speed_labels  intersection  weather  lighting  \
0              0             1          True        0         0   
1              0             1          True        0

In [218]:
null = data.isnull().sum()
null_sorted = null.sort_values(ascending = False)
print(null_sorted)
data = data.dropna()

severity            0
speed_labels        0
intersection        0
weather             0
lighting            0
traffic_controls    0
drugs_alcohol       0
pedestrian          0
inanimate           0
car                 0
large_vehicles      0
rail_vehicle        0
two_wheeled         0
dtype: int64


In [219]:
#data = DBSCAN(n_jobs = -2).fit(data).core_sample_indices_
#print(db)

In [220]:
X  = data.drop(columns = "severity")
Y = data['severity']

In [221]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 123, test_size = .3)

In [222]:
log_model = LogisticRegression()

In [223]:
scaler = RobustScaler()

In [224]:
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [225]:
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [226]:
X_train_final, y_train_final = SMOTE().fit_resample(X_train_scaled, y_train)
np.bincount(y_train_final)

array([189306, 189306, 189306], dtype=int64)

# **Log Reg**

In [227]:
steps =  [('pca', PCA(n_components=3)), ('m', LogisticRegression())]
model = Pipeline(steps = steps)

In [228]:
model.fit(X_train_scaled, y_train)

Pipeline(steps=[('pca', PCA(n_components=3)), ('m', LogisticRegression())])

In [229]:
y_pred_log = model.predict(X_test_scaled)

In [230]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.00      0.00      0.00      4898
           2       0.00      0.00      0.00       340

    accuracy                           0.94     86385
   macro avg       0.31      0.33      0.32     86385
weighted avg       0.88      0.94      0.91     86385



C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [231]:
def create_model(neurons= 5, optimizer = 'Adam', activation = 'sigmoid', layer_count = 2, learning_rate = .003):
  model = Sequential()    
  model.add(Dense(neurons, activation = activation, input_shape = (15, ), kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  for i in range(layer_count - 1):    
    model.add(Dense(neurons, activation=activation, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.L2(1e-4), activity_regularizer=regularizers.L2(1e-5)))
  model.add(Dense(1, activation='hard_sigmoid'))  
  model.compile(loss='categorical_crossentropy', optimizer= optimizer, metrics=['categorical_accuracy'])
  K.set_value(model.optimizer.learning_rate, learning_rate)
  return model

In [232]:
#nn = create_model()
#nn.fit(X_train_scaled, y_train, epochs = 200, verbose = 0)

In [233]:
#y_pred_net = nn.predict(X_test_scaled)

In [234]:
#y_pred_net = np.argmax(y_pred_net, axis=-1)

In [235]:
#print(classification_report(y_test, y_pred_net))

In [236]:
from sklearn.utils import resample

params = {
    'n_estimators' : [10, 50, 100, 200],
    'max_depth' : [4, 5, 10, 20, None],
    'max_features' : [None, 'sqrt', 'log2'],
    'min_impurity_decrease' : [0, 0.0005, 0.0007, 0.001, 0.01],
}
search = GridSearchCV(RandomForestClassifier(), params, n_jobs=-2, cv=5)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
model = search.fit(X_train_sampled, y_train_sampled)
best = model.best_estimator_.get_params()

In [237]:
rf = RandomForestClassifier(n_estimators = best['n_estimators'], 
                               max_depth = best['max_depth'],
                               max_features = best['max_features'], 
                               min_impurity_decrease = best['min_impurity_decrease'],
                               bootstrap = best['bootstrap'],
                               criterion = best['criterion']
                              )
rf.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(max_depth=20, min_impurity_decrease=0, n_estimators=50)

In [238]:
y_pred_rf = rf.predict(X_test_scaled)

In [239]:
#y_pred_rf = np.argmax(y_pred_rf, axis=-1)

In [240]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.35      0.03      0.06      4898
           2       0.17      0.03      0.05       340

    accuracy                           0.94     86385
   macro avg       0.49      0.35      0.36     86385
weighted avg       0.90      0.94      0.91     86385



In [241]:
dt = DecisionTreeClassifier()

In [242]:
dt.fit(X_train_scaled, y_train)

DecisionTreeClassifier()

In [243]:
y_pred_dt = dt.predict(X_test_scaled)

In [244]:
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.36      0.03      0.05      4898
           2       0.15      0.01      0.03       340

    accuracy                           0.94     86385
   macro avg       0.48      0.35      0.35     86385
weighted avg       0.91      0.94      0.91     86385



In [245]:
#ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
params = {
    "base_estimator" : [None, DecisionTreeClassifier()],
    "n_estimators" : [1, 25, 50, 100, 200, 500],
    "learning_rate" : [1.0, 0.1, 0.01, .001],
    "algorithm" : ["SAMME", "SAMME.R"]
}
search = GridSearchCV(AdaBoostClassifier(), params, n_jobs=-2, cv = 2)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
X_train_sampled, X_valid, y_train_sampled, y_valid = train_test_split(X_train_sampled, y_train_sampled, train_size=0.9)
model = search.fit(X_valid, y_valid)


best = model.best_estimator_.get_params()
ada = AdaBoostClassifier(base_estimator = best["base_estimator"],
                        n_estimators = best["n_estimators"],
                        learning_rate = best["learning_rate"],
                        algorithm = best["algorithm"])

In [246]:
ada.fit(X_train_sampled, y_train_sampled)

AdaBoostClassifier(algorithm='SAMME', n_estimators=1)

In [247]:
y_pred_ada = ada.predict(X_test_scaled)

In [248]:
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.00      0.00      0.00      4898
           2       0.00      0.00      0.00       340

    accuracy                           0.94     86385
   macro avg       0.31      0.33      0.32     86385
weighted avg       0.88      0.94      0.91     86385



C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [249]:
from sklearn.neighbors import KNeighborsClassifier

In [250]:
params = {
    "n_neighbors" : [2, 5, 7, 10],
    "metric" : ["manhattan", "minkowski"],
    "weights" : ["distance", "uniform"],
    "leaf_size" : [10, 15, 20, 25, 30, 50]
}
search = GridSearchCV(KNeighborsClassifier(), params, n_jobs=-2, cv = 3)
X_train_sampled, y_train_sampled = resample(X_train_scaled, y_train)
model = search.fit(X_train_sampled, y_train_sampled)
best = model.best_estimator_.get_params()

knn = KNeighborsClassifier(n_neighbors = best["n_neighbors"],
                           metric = best["metric"],
                           weights = best["weights"],
                           leaf_size = best["leaf_size"])
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(leaf_size=50, metric='manhattan', n_neighbors=10,
                     weights='distance')

In [251]:
y_pred_knn = knn.predict(X_test_scaled)

In [252]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.39      0.03      0.05      4898
           2       0.17      0.01      0.02       340

    accuracy                           0.94     86385
   macro avg       0.50      0.34      0.34     86385
weighted avg       0.91      0.94      0.91     86385



In [254]:
voter = VotingClassifier(estimators=[('adc', ada), ('rf', rf), ('knn', knn)])
voter.fit(X_train_scaled, y_train)
y_pred_voter = voter.predict(X_test_scaled)
print(classification_report(y_test,y_pred_voter))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     81147
           1       0.43      0.03      0.05      4898
           2       0.17      0.01      0.02       340

    accuracy                           0.94     86385
   macro avg       0.51      0.34      0.34     86385
weighted avg       0.91      0.94      0.91     86385

